### 第2章 构造数据抽象

摘要：
数据抽象的基本思想， 就是设法构造出一些使用复杂数据对象的程序，使她们就像是在“抽象数据”
上操作一样， 也就是说，她们的程序中使用数据的方式应该是这样的， 除了完成当前工作所必要的东西之外， 她们不对所用的数据做多余的假设， 于此同时， 一种“具体”数据表示的定义， 也应该与程序中使用数据的方式无关。 在我们的系统里， 这样两个部分之间的洁面将是一组过程， 称为`选择函数`和`构造函数`, 她们在具体表示之上实现抽象的数据。 

#### 有理数的表示

定义有理数
```
(define (make-rat n d) (cons n d))

(define (number x) (car x))

(define (denom x) (cdr x))
```

打印有理数

```
(define (print-rat x)
    (newline)
    (display (number x))
    (display "/")
    (display (denom x))
)
```

实验有理数

```
(define one-half (make-rat 1 2))

(print-rat one-half)

(one-half)

```


使用过程， 实现序列对

```
(define (cons x y)
    (define (dispatch m)
        (cond ( (= m 0) x)
              ( (= m 1) y)
              (else (error "Argument not 0 or 1 -- CONS" m))
        )
    )
    dispatch
)

(define (car z) (z 0))

(define (cdr z) (z 1))
```
测试使用
```
(car (cons 10 11))

(cdr (cons 10 11))
```

这段代码非常的精巧， 引用(cons 10 11)过程时，返回dispatch过程， 而car, cdr中是执行dispatch过程


 `练习题2.4`
 下面是序对的另一种过程性表示方式。请针对这一表示验证， 对于任意的x和y， (car (cons x y)) 都将产生出x
 
 ```
 (define (cons x y)
   (lambda (m) (m x y))
 )
 
 (define (car z)
   (z (lambda (p q) p))
 )
 
 (define (cdr z)
   (z (lambda (p q) q))
 )
 
 (car (cons 10 12))
 
 (cdr (cons 10 12))
 ```
 
 对应的cdr应该如何定义？ （提示： 为了验证这一表示确实能行， 请利用1.1.5章节的代换模型。
 ```
 (car (cons 10 12))
 
 替换cons=============
 
 (car (lambda (m) (m 10 12)))
 
 替换car==============
 ((lambda(m) (m 10 12) (lambda (p q) q)))
 
 替换lambda
 
 ((lambda (p q) p) 10 12)
 
 求解得出p
 
 还是非常精巧的设计
 
 ```

((lambda (p q) p) 10 12)
 
 

`练习题2.5`

请证明， 如果将a和b的序对表示为乘积 $
\begin{align}
2^a 3^b 
\end{align}
$
对应的整数， 我们就可以只用非负整数和运算进行序对。请给出对应的过程cons, car 和cdr的定义。

```
(define (cons x y)
    (* (expt 2 x)
       (expt 3 y)
    )
)

#主要整除2
(define (car z)
    (if (= 0 (remainder z 2))
        (+ 1 (car (/ z 2)))
        0
        
    )
)

#主要整除3
(define (cdr z)
    (if (= 0 (remainder z 3))
        (+ 1 (cdr (/ z 3)))
        0
        
    )
)
主要参考：https://sicp.readthedocs.io/en/latest/chp2/5.html


```

`练习题2.6`
如果觉得将序对表示为过程还不足以如雷灌耳， 那么考虑， 在一个可以对过程做各种操作的语言里， 我们完全可以没有数， 可以将0和加一操作实现为

```
(define zero (lambda (f) (lambda (x) x)))

(define (add-1 n)
    (lambda (f) 
        (lambda (x) (f ((n f) x)))
    )
)
这一表示形式称为Church计数， 名字来源其发明人数理逻辑学家Alonzo Church （丘奇）， lambda演算也是他发明的。

请直接定义one和two（不用zero和add-1), 提示：利用代换去求值(add-1 zero)).
请给出加法过程+的一个直接定义，(不要通过反复应用add-1

```

解题过程：

```

#+1过程
(add-1 zero)

#zero定义
(define zero (lambda (f) (lambda (x) x)))

#add-1定义
(define (add-1 n)
    (lambda (f) 
        (lambda (x) (f ((n f) x)))
    )
)

one = (add-1 zero)

(add-1 (lambda (f) (lambda (x) x)))

#代换01
(lambda (f) 
    (lambda (x) (f ((lambda (f) (lambda (x) x) f) x)))
)

#代换02
(lambda (f) 
    (lambda (x) (f ((lambda (f) f) x)))
)

#代换03
(lambda (f) 
    (lambda (x) (f x))
)

即
(define one
    (lambda (f) 
        (lambda (x) (f x))
    )
)


#+2过程

two = (add-1 one)

#add-1定义
(define (add-1 n)
    (lambda (f) 
        (lambda (x) (f ((n f) x)))
    )
)


(add-1 
   (lambda (f) 
        (lambda (x) (f x))
    )
)

#代换01
#代换比较绕todo

参考：https://sicp.readthedocs.io/en/latest/chp2/6.html

```

`2.1.4区间算数`

区间加法运算

```
(define (add-interval x y)
    (make-interval (+ (lower-bound x) (lower-bound y))
                   (+ (upper-bound x) (upper-bound  y))
    )
```

区间乘法运算

```
(define (mul-interval x y)
  (let (
         (p1 (* (lower-bound x) (lower-bound y)))
         (p2 (* (lower-bound x) (upper-bound y)))
         (p3 (* (high-bound x)  (lower-bound y)))
         (p4 (* (high-bound x)  (high-bound y)))
       )
     (make-interval (min p1 p2 p3 p4)
                    (max p1 p2 p3 p4)
     )
  )

)
```

区间除法， 第一个区间乘上第二个区间的倒数， 请注意， 倒数的两个限界分别是原来区间的上界的倒数和下界的倒数。

```
(define (div-interval x y)
    (mul-interval x
                  (make-interval (/ 1.0 (upper-bound y))
                                 (/ 1.0 (lower-bound y))
                  )
    )
)
```

`练习题2.7`

程序不完整， 因为还没有确定区间抽象的实现， 这里的区间构造符的定义：

```
(define (make-interval a b) (cons a b))
```

请定义选择符upper-bound 和lower-bound, 完成这一实现

```
(define (upper-bound z)
    (cdr z)
)

(define (lower-bound z)
    (car z)
)
```

`练习题2.8`

通过推理， 请定义处相应的减法过程 sub-interval.

max区间：
先定义大减小------


疑惑点这里是  x-y, 不需要考虑y-x
```
(define (sub-interval x y)
    (make-interval (- (lower-bound x) (upper-bound y))
                   (- (high-bound x) (lower-bound  y))
    )
```


#### 2.2 层次性数据和闭包性质

(list a1 a2 ....an) 等价于

(cons a1 (cons a2 (cons .....(cons an nil)...)))

(define one-through-four (list 1 2 3 4))

表达式(list 1 2 3 4)和表(1 2 3 4)是两个概念， 后面这个表是对前面表达式求值得到的结果。


nil的值用于表示序对的链结束


#### 表操作

定义 list-ref  参数一个表 和一个数n, 它返回这个表中的第n个项， 这里人们习惯令元素的编号从0开始， 计算list-ref的方法如下

对n= 0, list-ref 返回表的car

否则， list-ref返回表的cdr的第(n-1)个项

```
#根据下标获取对应的值
(define (list-ref items n)
  (if (= n 0)
      (car items)
      (list-ref (cdr items) (- n 1))
  )
)

#测试例子
(define squares (list 1 4 9 16 25))

(list-ref squares 3)
```

```
#返回list总长度

(define (length items)
   (if (null? items)
       0
       (+ 1 (length (cdr items)))
   )
)

#测试例子
(define odds (list 1 3 5 7 8))

(length odds)

过程length实现的一种简单的递归方案， 其中的递归步骤是:

1. 任意一个表的length就是这个表的cdr的length加一
顺序的这样应用， 直至达到了基础情况：
2. 空表的length是0

我们也可以采用一种迭代的方式来计算length:

(define (length items)
    (define (length-iter a count)
        (if (null? a)
            count
            (length-iter (cdr a) (+ 1 count))
        )
    )
    (length-iter items 0)
)

#测试用例子
(length (list 1 2 3 4))

```

append也是用一种递归方案实现。 要得到表list1 和list2的append, 按如下方式做：

```

1. 如果list1是空表，结果解释lsit2.
2. 否则应先做处list1的cdr和list2的append, 而后再将list1的car通过cons驾到结果的前面

#append定义

(define (append list1 list2)
   (if (null? list1)
       list2
       (cons (car list1) (append (cdr list1) list2))
   )
)

#测试相关

(append (list 1 2 3) (list 2 3 4))

```

`练习题2.17`

请定义处过程last-pair, 它返回只包含给定表立最后一个元素的表：

```
(define (last-pair a)
    (if (null? (cdr a))
        (car a)
        (last-pair (cdr a))
    )
)
```

#例子
(last-pair (list 23 72149 34 1111))


`练习题2.18` 请定义处过程reverse, 它以一个表为参数， 返回的表中包含的元素与参数表相同，但排列顺序与参数表相反：

(reverse (list 1 4 9 16 25))


```
;递归方式执行reverse, 但是必须引入中间append, 不是太优雅
(define (reverse a)
    (if (null? (cdr a))
        (list (car a))
        (append (reverse (cdr a)) (list (car a)))
    )
    
)

;采用其他方式, 这种不合适， cons 只能执行 (cons variable list)格式
(define (reverse a)
  (if (null? (cdr a))
        (car a)
        (cons (reverse (cdr a)) (car a))
  )
)

;迭代方式执行reverse
(define (reverse a)
    ;定义空list
    (define result
        '()
    )
    (define (new-reverse a result)
        (if (null? a)
            result
            (new-reverse (cdr a) (cons (car a) result))
        )
    )
    (new-reverse a result)
)
```

`练习题2.19` 请考虑 1.2.2章节的兑换零钱方式计数程序， 如果能够轻而易举地改变程序里面的兑换币种就更好，比如，我们就能够计算出1英镑的不同兑换方式的数目， 在写这个程序之前， 有关币种的知识有一部分出现在过程first-denomination里面， 另一部分出现在过程count-change(它知道有5种us硬币). 如果能偶用一个表来提供可用于兑换的硬币就好了。
    我们希望重写出过程cc, 使其第二个参数是一个可用硬币的币值表， 而不是一个指定可用硬币种类的整数。 而后我们就可以针对各种货币定义一些表：
```
(define us-coins (list 50 25 10 5 1))

(define uk-coins (list 100 50 20 10 5 2 1 0.5))

然后我们预期通过如下方式调用过程cc

(cc 100 us-coins)

为了做到这件事， 我们需要对程序cc做一些修改， 它仍然具有同样的形式， 但将以不同的方式访问自己的第二个参数， 如下面所提示：

(define (cc amount coin-values)
    (cond ((= amount 0) 1)
          ((or (< amount 0) (no-more coin-values)) 0)
          (else
              (+ (cc amount (except-first-denomination coin-values))
                 (cc (- amount (first-denomination coin-values))
                     coin-values
                 )
              )
          )
    )
)
请基于表结构上的基本操作， 定义出过程 first-denomination, except-first-denomination 和 no-more, 表coin-values的排列顺序会影响cc给出的答案吗？为什么？

#解题如下：

核心问题是支持兑换多币种。核心算法没有变化

;校验表长度
(define (no-more coin-values)
    (> 1 (length coin-values))
)

;获取首个元素值
(define (first-denomination coin-values)
    (car coin-values)
)

;排除第一个元素
(define (except-first-denomination coin-values)
    (cdr coin-values)
)

```

`练习题2.20` 过程 +,*和list可以取任意个数的实际参数。 定义这类过程的一种方式是：采用一种带点尾部计法形式的define, 在一个过程定义中， 如果在形式参数表的最后一个参数之前有一个点号。 那就表明 当这一过程被实际调用时，前面各个形式参数，将以前面的各个实际参数为值， 与平常一样。 但最后一个形式参数，将以所有剩下的实际参数的表为值， 

例如， 我们定义了：

(define (f x y . z) <body>)
    
过程f就可以用两个以上的参数调用， 如果求值:

(f 1 2 3 4 5 6)

那么在f的体内， x将是1， y将是2， 而z将是表(3 4 5 6), 给了定义：

(define (g . w) <body>)

过程g可以用0个或多个参数调用， 如果求值：

(g 1 2 3 4 5 6)

那么在g的过程里面， w将是表(1 2 3 4 5 6).

请采用这种记法写出过程 same-parity, 它以一个或者多个整数为参数， 返回所有与其第一个参数有着同样奇偶性的参数形式的表， 例如：

```

(same-parity 1 2 3 4 5 6 7)
(1 3 5 7)

(same-parity 2 3 4 5 6 7)
(2 4 6)




; 迭代方式，运行， 这里后期引入filter效果更加， 学完filter补充，使用 filter格式
(define (same-parity a . w)
    (define (num_type a)
        (remainder a 2)
    )
    (define (iter-result new-w result1)
        (if (null? new-w)
            result1
            (iter-result (cdr new-w)
                (if (= (num_type a) (remainder (car new-w) 2))
                       (cons (car new-w) result1)
                       result1
                   )
            )
        )
    )
    (define result
        (list a)
    )
    (iter-result w result)
    
)
```

#### 对表的映射

一个特别有用的操作是将某种变换应用于一个表的所有元素， 得到所有结果构成的表。 举例来说， 下面过程将一个表里的所有元素按给定因子，做一次缩放：

```
(define (scale-list items factor)
    (if (null? items)
        '()
        (cons (* (car items) factor)
              (scale-list (cdr items) factor)
        )
    )
)

;验证测试
(scale-list (list 1 2 3 4 5) 10)
```

我们可以抽象出这一具有一般性的思想， 将其中的公共模式表达为一个高阶过程， 就像1.3 章节里所做的那样， 这一高阶过程称为map, 它有一个过程参数和一个表参数， 返回将这一过程应用于表中各个元素得到的结果形成的表

```
(define (map proc items)
  (if (null? items)
      '()
      (cons (proc (car items))
            (map proc (cdr items))
      )
  )
)
```

;测试验证：
(map abs (list -10 2.5 -11.6 17))

;继续测试验证
(map (lambda (x) (* x x))
    (list 1 2 3 4)
)

现在我们可以用map给出scale-list的一个新定义:

```
(define (scale-list items factor)
    (map (lambda (x) (* x factor))
         items
    )
       
)
```

map是一种很重要的机构， 不仅因为它代表里一种公共模式， 而且因为它建立起里一种处理表的高层抽象， 在scale-list原来的定义里， 程序的递归结构将人的注意力吸引到对表中逐个元素的处理上。 通过map定义scale-list抑制了这种细节层面的情况， 强调了是从元素表到结果表的一个缩放转换。 这两种定义形式之间的差异， 并不在于计算机会执行不同的计算过程， 而在于我们对于同一个过程的不同思考方式。 从作用上来看， map帮我们建立了一层抽象屏障， 将实现表变换转换的过程实现， 与如何提取表中元素及组合结果的细节隔离开。 这种抽象提供了新的灵活性， 使我们有可能在保持从序列到序列的转换操作框架的同时， 改变序列实现的底层细节。

`练习2.21` 过程square-list以一个数值表为参数， 返回每个数的平方构成的表：

(square-list (list 1 2 3 4))

下面是square-list的两个定义，请填充其中缺少的表达式，以完成它们：

```
(define (square-list items)
    (if (null? items)
        '()
        (cons (* (car items) (car items))
            (square-list (cdr items))
        )
    )
)

(define (square-list items)
    (map (lambda (x) (* x x))
        items
    )
)

```

`练习题2.22` 重写square-lsit过程， 希望它能生成一个迭代计算过程

```
(define (square-list items)
  (define (iter things answer)
     (if (null? things)
         answer
         (iter (cdr things)
               (cons (square (car things))
                     answer
               )
         )
     )
  )
  (iter items '())
)
```

;过程相反
(square-list (list 1 2 3 4))


```
(define (square-list items)
  (define (iter things answer)
     (if (null? things)
         answer
         (iter (cdr things)
               (cons answer
                     (square (car things))
               )
         )
     )
  )
  (iter items '())
)
```
交换顺序，之后，嵌套， 不满足要求，因为cons 只支持(cons element (list xxx))
(square-list (list 1 2 3 4))

```
14 error> (square-list (list 1 2 3 4))

;Value 27: ((((() . 1) . 4) . 9) . 16)
```


可以考虑引入append

```
(define (square-list items)
  (define (iter things answer)
     (if (null? things)
         answer
         (iter (cdr things)
               (append answer
                       (list (square (car things)))
               )
         )
     )
  )
  (iter items '())
)
```

;验证如下：
(square-list (list 1 2 3 4))

```
14 error> (square-list (list 1 2 3 4))

;Value 28: (1 4 9 16)
```


`练习题2.23` 过程for-each与map类似， 它以一个过程和一个元素表为参数， 但它并不返回结果的表， 知识将这一过程从左到右应用于各个元素， 将过程应用于元素得到的值都丢掉。 for-each通常用于哪些执行了某些动作的过程， 如打印等， 如看下下面的例子：

```
(for-each (lambda (x) (newline) (display x))
          (list 57 321 88)
)
```

预期输出：
57
321
88

```
;这里引入begin命令， 使if支持多条语句
(define (for-each proc items)
  (if (null? items)
      '()
      (begin
          (proc (car items))
          (for-each proc (cdr items))
      )
  )
)

;验证

;可以引入 cond， cond默认支持多条语句， 验证，完全支持， 这里隐藏使用(begin ,  

(define (for-each proc items)
   (cond ((null? items) '())
         (else
             (proc (car items))
             (for-each proc (cdr items))
         )
   )
)
参考： https://sicp.readthedocs.io/en/latest/chp2/23.html
```





#### 2.2.2 层次性结构


递归是处理树结构的一种很自然的工具， 因为我们常常可以将对于树的操作归结为对它们分支操作
， 再将这种操作归结为对分支的分支的操作， 如此下去，直至到达树的叶子。

(define x (cons (list 1 2) (list 3 4)))

3

(count-leaves x)

4

为了实现count-leaves, 可以先会议一下length的递归方案：

* 表x的length是x的cdr的length加一
* 空表的length是0

count-leaves的递归方案与此类似， 对于空表的值也相同：
* 空表的count-leaves是0

但是在递归步骤中，当我们去掉一个表的car时， 就必须注意这一car本身也可能是树， 其树叶也应该考虑， 正确的规约步骤应该是:

* 对于树x的count-leaves应该是x的car的count-leaves与x的cdr的count-leaves只和

最后，在通过car达到一个实际的树叶时， 我们还需要提供一种基本情况：

* 一个树叶的count-leaves是1

为了有助于写出树的各种递归， Scheme提供了基本过程 `pair?`, 它检查其参数是否为序对， 下面就是我们完成的过程：

```
(define (count-leaves x)
    (cond ((null? x) 0)
          ((not (pair? x)) 1)
          (else 
              (+ (count-leaves (car x))
                  (count-leaves (cdr x))
              )
          )
    )
    
)
```

验证
```
(count-leaves x)
```

这里的重点是引入pair的概念：

`练习题2.24`

假定现在求值表达式 (list 1(list 2 (list 3 4))) 请给出由解释器打印出的结果， 给出对应的盒子指针结构， 并且解释为一颗树

比较简单， 手动操作即可

`练习题 2.25`

给出讷讷狗从下面各表中取出7的car 和cdr组合

```
(1 3 (5 7) 9)
;执行结果
(define x1 (list 1 3 (list 5 7) 9))
(car (cdr (car (cdr (cdr x1)))))

((7))
;执行结果
(define x2 (list (list 7)))
(car (car x2))

(1 (2 (3 (4 (5 (6 (7)))))))
;执行结果
(define x3 (list 1 (list 2 (list 3 (list 4 (list 5 (list 6 (list 7))))))))

(car (car (cdr (car (cdr (car (cdr (car (cdr (car (cdr (car (cdr x3)))))))))))))

这个现象比较特殊,  跟想象中不一样， 需要交替car, cdr

```


`练习题2.26` 假定已将x和y定义为如下的两个表：

(define x (list 1 2 3))
(define y (list 4 5 6))

解释器对于下面各个表达式将打印出什么结果

(append x y)

(1 2 3 4 5 6)

(cons x y)

((1 2 3) 4 5 6)

(list x y)

((1 2 3 ) 4 5 6)

```
实际结果：

(list x y)跟想象的结果不一样

((1 2 3) (4 5 6)), 这里(4 5 6)会占一个节点


`练习题 2.27` 修改练习题2.18中所做的reverse过程， 得到一个deep-reverse过程， 它可以返回另一个表作为值， 结果表中的元素翻转过来， 其中的子树也翻转， 例如

(define x (list (list 1 2) (list 3 4)))

(reverse x)
预期
((3 4) (1 2))

(deep-reverse x)

预期
((4 3) (2 1))


;直接参考的https://sicp.readthedocs.io/en/latest/chp2/27.html
(define (deep-reverse tree)
    (cond ((null? tree)         ; 空树
            '())
          ((not (pair? tree))   ; 叶子
            tree)
          (else
            (reverse (list (deep-reverse (car tree))            ; 递归地逆序左右子树
                           (deep-reverse (cadr tree)))))))
                          
这里先逆序， 后调整， 这里主要依赖reverse



```


`练习题2.28` 写一个过程fringe ,它以一个树(表示为表）作为参数，返回一个表， 表中的原孙是这棵树的所有树叶， 按照从左到右的顺序， 例如

 ```
 (define x (list (list 1 2) (list 3 4)))
 
 (fringe x)
 
 (fringe (list x x))
 
 (1 2 3 4 1 2 3 4)
 
 (define (fringe x)
     (cond
          ((null? x) '())
          ((not (pair? x)) (list x))
          (else 
              (append (fringe (car x)) (fringe (cadr x)))
          )
     )
 )
 ```

`练习题2.29` 一个二叉活动体由两个分支组成， 一个是左分支， 另一个是右分支， 每个分支是一个具有确定长度的杆， 上面或者吊着一个重量， 或者吊着另一个二叉活动提， 我们可以用复合数据对象表示这种二叉活动体， 将它通过两个分支构造起来，例如 使用list

```
(define (make-mobile left right)
    (list left right)
)
```

分支可以从一个length(表示一个数)  再加上一个structure构造出来， 这个structure或者是一个树， 或者另一个活动体:

```
(define (make-branch length structure)
    (list length structure)
)
```

a) 请写出相应的选择函数left-branch和right-branch, 它们分别返回活动体的两个分支。 还有branch-length和branch-structure, 它们返回一个分支上的成分。

```
左分支
(define (left-branch tree)
    (car tree)
)

右分支
(define (right-branch tree)
    (cadr tree)
)

#总量,总长度
(define (branch-length branch)
    (car branch)
)

#结构体-二叉活动体
(define (branch-structure branch)
    (cadr branch)
)
```

验证
(define mobile (make-mobile (make-branch 10 25)
                                  (make-branch 5 20)))
```
(define mob1
  (make-mobile (make-branch 3 6)
	       (make-branch 2 9)))

(define mob2
  (make-mobile (make-branch 6 4)
	       (make-branch 5 mob1)))

(define mob3
  (make-mobile (make-branch 15 3)
	       (make-branch 3 mob1)))

(branch-structure (right-branch mob1))
(branch-structure (right-branch mob2))
```
                                  

b)用你的选择函数定义过程total-weight, 它返回一个活动体的总重量。

主要是规约两点：
1. 总重量等于左右分支 重量之和
2. 如果是一个数字，则代表是重量

```
(define (total-weight tree)
    (cond ((not (pair? tree)) tree)
          (else 
              (+ (total-weight (branch-structure (left-branch tree)))
                 (total-weight (branch-structure (right-branch tree)))
              )
          )
        
    )
    
)

#验证：
 (define mobile (make-mobile (make-branch 10 25)
                                  (make-branch 5 20)))
                                  
(define a (make-mobile (make-branch 2 3) (make-branch 2 3)))

(total-weight a)
```

c) 一个活动体是平衡的， 如果其左分支的力矩等于其右分支的力矩（也就说，如果其左杆的长度乘以吊在杆上的重量， 等于这个活动体右边的同样乘机） 而且在每个分支上吊着子活动体也都平衡， 请设计一个过程， 它能检查一个二叉活动体是否平衡

```
规约核心点：
1. 力矩 = 长度 * 重量
2. 每个分支上吊着的活动体也都平衡

(define (balanced? tree)
    (= (* (branch-length (left-branch tree))
          (total-weight (branch-structure (left-branch tree)))
       ) 
       
       (* (branch-length (right-branch tree))
          (total-weight (branch-structure (right-branch tree)))
       )
       
    )
)


;网上参考
(define (balanced? mobile)
  (let ((left  (left-branch  mobile))
       (right (right-branch mobile))
	   (b-struct branch-structure)
	   (b-length branch-length))
    (= (* (total-weight (b-struct left))
	      (b-length left))
       (* (total-weight (b-struct right))
	      (b-length right)))))
      

```

验证： (balanced? mobile)


d) 假设我们改变活动体的表示，采用下面的构造方式：

```
(define (make-mobile left right)
    (cons left right)
)

(define (make-branch length structure)
    (cons length structure)
)

#左分支
(define (left-branch tree)
    (car tree)
)

#右分支
(define (right-branch tree)
    (cdr tree)
)

#总量,总长度
(define (branch-length branch)
    (car branch)
)

#结构体-二叉活动体
(define (branch-structure branch)
    (cdr branch)
)


```


你需要对自己的程序做多少修改， 才能将它改为使用这种新表示？

这里主要是考cons 和list区别




#### 对树的映射


map是处理序列的一种强有力的抽象，与此类似， map与递归的结合也是处理树的一种强有力抽象，举例来说， 可以于2.2.1节的scale-list类似的scale-tree过程， 以一个数值因子和一颗叶子为数值的树作参数， 返回一颗具有同样形状结构的树， 树中的每个数值都相相乘， 对于scale-tree的递归方案也与count-leaves的类似：

```
(define (scale-tree tree factor)
    (cond ((null? tree) '())
          ((not (pair? tree)) (* tree factor))
          (else
              (cons (scale-tree (car tree) factor)
                    (scale-tree (cdr tree) factor)
              )
          )
         
    )
)

(define new-tree (list 1 (list 2 (list 3 4) 5) (list 6 7)))

;验证

(scale-tree new-tree 2)
```

实现scale-tree的另一种方法是将树堪称子树的序列， 并对它使用map。 我们在这种序列上做映射， 依次对各科子树做缩放， 并返回结果的表。 对于基础情况， 也就是当被处理的树是树叶时， 就直接用因子去乘它：

```
(define (scale-tree tree factor)
    (map (lambda (sub-tree)
           (if (pair? sub-tree)
               (scale-tree sub-tree factor)
               (* sub-tree factor)
           )
         )
        tree
    )
)
```

;验证

(scale-tree new-tree 2)




`练习2.30` 请递归一个与练习2.21中 square-list 过程类似的square-tree过程， 也就是说， 它应该具有下面的行为:

```
(define new-tree (list 1 (list 2 (list 3 4) 5) (list 6 7)))
(1 (4 (9 16) 25) (36 49))

```


```
;方法1
(define (square-tree tree)
    (cond ((null? tree) '())
          ((not (pair? tree)) (* tree tree))
          (else
              (cons (square-tree (car tree))
                    (square-tree (cdr tree))
              )
          )
    )
)
```
验证

```
(square-tree new-tree)

;方法2, 遇到的问题是忘了加上map

(define (square-tree tree)
     (map (lambda (sub-tree)
             (if (not (pair? sub-tree))
                 (* sub-tree sub-tree)
                 (square-tree sub-tree)

             )
         )
         tree
     )
)

```


`练习题2.31` 将你在练习2.30作出的解答进一步抽象， 作出一个过程， 使他的性质保证能以下面形式定义square-tree:

```
(define (square-tree tree) (tree-map square tree))

(define (tree-map proc tree)
    (map (lambda (sub-tree)
            (if (not (pair? sub-tree))
                (proc sub-tree)
                (tree-map proc sub-tree)
            )
         )
         tree
    )
)

(square-tree new-tree)

```

`练习2.32` 我们可以将一个集合表示为一个元素互不相同的表， 因此就可以将一个集合的所有子集表示为表的表。 例如， 假定集合为(1 2 3)，它的所有子集的结合就是
(() (3) (2) (2 3) (1) (1 3) (1 2) (1 2 3)), 请完成下面的过程定义， 它生成一个集合的所有子集的集合。 请解释它为什么能完成这一工作。。

```
(define (subsets s)
   (if (null? s)
       (list '())
       (let (
              (rest (subsets (cdr s)))
            )
            (append rest 
                    (map (lambda (sub-tree) 
                                (cons (car s) sub-tree)
                         ) 
                          rest
                    )
            )
       )
   )
)

;验证：

(subsets (list 1 2 3))

参考： https://sicp.readthedocs.io/en/latest/chp2/32.html

需要重新理解

和兑换零钱规则一样
主要以下规约：
1. 全部子集 = 不包含第一个元素element的子集A + 包含第一个元素的子集B(即其所有子集列表A都要包含element 即是子集B)
2. 子集为空， 则为空列表

英文参考：
http://www.billthelizard.com/2011/03/sicp-232-generating-power-sets.html


```

#### 2.2.3  序列作为一种约定的字段

要组织好这些程序， 使之能够更清晰地反应上面信号流的结构， 最关键的一点就是将注意力集中处理过程中从一个步骤流向下一个步骤的“信号”， 如果我们可以用一些表来表示这些信号，那么就可以利用表操作实现每一步骤的处理。 举例来说， 我们可以用2.2.1章节的map过程来实现信号流图中的映射步骤：

(map square (list 1 2 3 4 5))

过滤一个序列， 也就是选出其中满足某个未定谓词的元素， 可以按下面方式做：

```
;定义过滤操作
(define (filter predicate sequence)
   (cond ((null? sequence) '())
         ((predicate (car sequence))
            (cons (car sequence) (filter predicate (cdr sequence)))
         )
         (else 
            (filter predicate (cdr sequence))
         )
   )
)

;验证
(filter odd? (list 1 2 3 4 5))
;Value 3: (1 3 5)
```

```
;定义累积工作
;op操作类型(* +, 还是其他)
;initial初始值
;sequence默认值
(define (accumulate op initial sequence)
  (if (null? sequence)
      initial
      (op (car sequence)
          (accumulate op initial (cdr sequence))
      )
  )
)

验证
(accumulate + 0 (list 1 2 3 4 5))
```
对于even-fibs 我们需要生成一个给定区间里的整数序列， 这一序列可以如下做出：

```
(define (enumerate-interval low high)
   (if (> low high)
       '()
       (cons low (enumerate-interval (+ low 1) high))
   )
)

;验证
(enumerate-interval 2 7)

要枚举出一棵树的所有树叶， 则可以用:

(define (enumberate-tree tree)
  (cond ((null? tree) '())
        ((not (pair? tree)) (list tree))
        (else 
           (append (enumberate-tree (car tree))
                   (enumberate-tree (cdr tree))
           )
        )
  )
)

;验证
(enumberate-tree (list 1 (list 2 (list 3 4)) 5))
```



现在， 我们就可以像上面的信号流图那样重新构造 sum-odd-squares和even-fibs

```
(define (sum-odd-squares tree)
  (accumulate +
              0
              (map square
                   (filter odd?
                           (enumerate-tree tree)
                   )
              )
  )
)
```

将程序表示为一些针对序列的操作， 这样做的价值就在于能帮助我们得到模块化的程序， 也就是说，得到由一些比较独立的片段的组合构成的设计， 通过提供一个标准部件库， 使这些部件都有一些能以各种灵活方式相互连接的约定界面， 将能进一步推动人们去做模块化的设计。

在这里， 用表实现的序列被作为一种方便的界面， 我们可以利用这种界面去组合起各种处理模块。 进一步说， 如果以序列作为所用的统一表示结构， 我们就能将程序对于数据结构的依赖性局限到不多的几个序列操作上。 通过修改这些操作。 就可以在序列的不同表示之间转换， 并保持程序的整合设计不变。


`练习2.33` 请填充下面缺失的表达式， 完成将一些基本的表操作看作累积的定义：

```
(define (map p sequence)
  (accumulate (lambda (x y) (cons (p x) y ) ) '()        
              sequence
  )
)

;第一版错误
(define (append seq1 seq2)
  (accumulate cons '() (cons seq1 seq2))
)
;正确方法
(define (append seq1 seq2)
    (accumulate cons seq2 seq1))

(define (length sequence)
  (accumulate (lambda (x y) (+ 1 y)) 0 sequence)
)
```

`练习题2.34` 对于x的某个给定值， 求出一个多项式在x的值， 也可以形式化为一种累积，假定需要求下面多项式的值


$a{_n}x{^n}+a{_{n-1}}x{^{n-1}}+...+a{_1}x+a{_0}$

采用著名的Horner规则， 可以构造出下面的计算：

$ (···(a{_n}x+a{_{n-1}})x+···+a{_1})x + a{_0}$

换句话， 我们可以从$a{_n}$开始， 乘以x, 再加上$a{_{n-1}}$, c乘以x, 如此下去， 直到处理完$a{_0}$, 请填充下面的模版， 作出一个利用Horner规则求多项式值的过程。 假定多项式的系数安排在一个序列里， 从$a{_0}$直至$a{_n}$

```
(define (horner-eval x coefficient-sequence)
  (accumulate (lambda (this-coeff higher-terms) 
                      (
                          + this-coeff (* x higher-terms)
                      ) 
              )
              0
              coefficient-sequence
  )
)
```

例如, 为了计算$1+3x+5x{^3}+x{^5}$ 在x=2的值， 你需要求值

```
(horner-eval 2 (list 1 3 0 5 0 1))
```


`练习2.35` 将2.2.2节的count-leaves重新定义为一个累积：

```
;最开始定义的count-leaves
(define (count-leaves x)
    (cond ((null? x) 0)
          ((not (pair? x)) 1)
          (else 
              (+ (count-leaves (car x))
                  (count-leaves (cdr x))
              )
          )
    )

)

(define (count-leaves t)
  (accumulate + 0 (map (lambda (x) (+ ) ) t))
)

;参考结果：
(define (count-leaves tree)
    (accumulate +
                0
                (map (lambda (sub-tree)
                         (if (pair? sub-tree)           ; 如果这个节点有分支
                             (count-leaves sub-tree)    ; 那么这个节点调用 count-leaves 的结果就是这个节点的树叶数量
                             1))                        ; 遇上一个叶子节点就返回 1
                     tree)))
                     
```


`练习题2.36` 过程accumulate-n与accumulate类似， 除了它的第三个参数是一个序列的序列， 假定其中每个序列的元素个数相同， 它用指定的累积过程去组合起所有的序列的第一个元素， 而后是所有的序列的第二个元素， 并如此做下去， 返回得到的所有结果的序列， 例如。 如果s是包含着4个序列的序列((1 2 3) (4 5 6) (7 8 9) (10 11 12)) 那么 (accumulate-n + 0 s)的值就应该是序列(22 26 30) ，请填充下面的accumulate-n定义中所缺失的表达式：

```
(define (accumulate-n op init seqs)
  (if (null? (car seqs))
      '()
      (cons (accumulate op init <??>)
            (accumulate-n op init <??>)
      )
  )
)
```

```
;结果, 犯错：没有仔细审题
(define (accumulate-n op init seqs)
  (if (null? (car seqs))
      '()
      (cons (accumulate op init (map car seqs))
            (accumulate-n op init (map cdr seqs))
      )
  )
)


;验证

(define x (list (list 1 2 3) (list 4 5 6) (list 7 8 9) (list 10 11 12)))

(accumulate-n + 0 x)
```



`练习题2.371`

`练习题2.38` 过程accumulate也称为fold-right, 因为它将序列的第一个元素组合到右边所有元素的组合结果上。 也有一个fold-left, 它与fold-right类似， 但却是按照反方向去操作各个元素：


```
(define (fold-left op initial sequence)
  (define (iter result rest)
     (if (null? rest)
       result
       (iter (op result (car rest))
             (cdr rest)
       )
     )
  )
  (iter initial sequence)
)


```

表达式验证：

```
(fold-right / 1 (list 1 2 3))

(fold-left /  1 (list 1 2 3))

(fold-right list '() (list 1 2 3))

(fold-left list '() (list 1 2 3))
```

如果要求用某个op时保证fold-right和fold-left对任何序列都产生同样的结果， 请给出op应该满足的性质

```
(fold-right + 0 (list 1 2 3))
(fold-left + 0 (list 1 2 3))
```

需要复合结合律

```
像 + 、 * 、 or 和 and 那样的函数，就是符合结合律的函数
```






`练习题2.39`

基于练习2.38的fold-right和fold-left完成 reverse(练习题2.18) 下面的定义：

需要满足结合律

```
(define (reverse sequence)
  (fold-right (lambda (x y) (append y (list x))) '() sequence)
)



(define (reverse sequence)
  (fold-left (lambda (x y) (cons y x)) '() sequence)
)
```

验证

```
(reverse (list 1 2 3 4 5))
```

#### 嵌套映射

我们可以扩充序列模型， 将许多常用嵌套循环表述的计算也包含进来， 现在考虑以下的问题： 给定来自然数n， 找出所有不同的有序对i和j, 其中 1<= j <i <= n 使得i+j是素数， 例如， 假定n是6， 满足条件的需对就是


完成这一计算的一种很自然的组织方式： 首先生成所有小于等于n的正自然数的有序对； 而后通过过滤， 得到那些和为素数的有序对， 最后对每个通过来过滤的序对(i, j)产生出一个三元组 (i, j, i+j)

这里是生成的有序对的序列的一种方式： 对于每个整数i<=n, 枚举出所有的整数 j<i, 并对每一对i和j生成序对 (i, j) ；用序列操作的方式说， 我们要对序列(enumerate-interval 1 n) 做一次映射。 对于这个序列里的每个i, 我们都要对序对(enumerate-interval 1 (- i 1))做映射， 对于后一序列中的每个j， 我们生成出序对(list i j), 这样就对每个i 得到了一个序对的序列。 将针对所有i的序列组合到一起(用append累积起来)就能产生出所需要的序对序列了

```
(define (enumerate-interval low high)
   (if (> low high)
       '()
       (cons low (enumerate-interval (+ low 1) high))
   )
)

(define (accumulate op initial sequence)
  (if (null? sequence)
      initial
      (op (car sequence)
          (accumulate op initial (cdr sequence))
      )
  )
)


(accumulate append
            '()
            (map (lambda (i)
                     (map (lambda (j) (list i j))
                          (enumerate-interval 1 (- i 1))
                     )
                 )
                 (enumerate-interval 1 6)
            )
)
```

由于这类程序经常要用到映射， 并用append做累积， 我们将它独立出来定义一个过程

```
(define (flatmap proc seq)
  (accumulate append '() (map proc seq))
)
```


现在可以过滤这一序对的序列， 找出那些和为素数的序对， 对序列里的每个元素调用过滤谓词， 由于这个谓词的参数是一个序对， 所以它必须将两个整数从序对里提取出来。 这样， 作用到序列中的每个元素上的谓词就是：


```
;过滤 这里校验素数， 复用的第一张定义的素数校验函数
(define (prime-sum? pair)
   (prime? (+ (car pair) (cadr pair)))
)

(define (prime? n)
    (= n (smallest-divisor n)))

(define (smallest-divisor n)
    (find-divisor n 2))

(define (find-divisor n test-divisor)
    (cond ((> (square test-divisor) n)
            n)
          ((divides? test-divisor n)
            test-divisor)
          (else
            (find-divisor n (+ test-divisor 1)))))

(define (divides? a b)
    (= (remainder b a) 0))
    
```

最后还要生成出结果的序列， 为此只需将下面过程映射到通过过滤后的序对上， 对每个有序对里的两个元素， 这一过程生成出一个包含里它们的和的三元组

```
;拼装结果数据
(define (make-pair-sum pair)
  (list (car pair) (cadr pair) (+ (car pair) (cadr pair)))
)
```

将这些组合到一起， 就得到了完整的过程

```
(define (prime-sum-pairs n)
  (map make-pair-sum
       (filter prime-sum?
               (flatmap
                 (lambda (i)
                   (map (lambda (j) (list i j))
                        (enumerate-interval 1 (- i 1))
                   )
                 )
                 (enumerate-interval 1 n)
               )
       )
  )
)
```

验证：

```
(prime-sum-pairs 6)
```


嵌套的映射不仅能用于枚举这种区间， 也可以用于其他序列。 假设我们希望生成集合S的所有排序， 也就是说， 生成这一集合的元素的所有可能排序方式。 例如{1, 2, 3}的所有排列是 {1, 2, 3}, {1, 3, 2}, {2, 1, 3}, {2, 3 1}, {3, 1, 2}和 {3, 2, 1} 这里生成S的所有排序的一种方案是：



对于S里的每个x， 递归的生成S-x的所有排列的序列， 而后将加到每个序列的前面， 这样就能对S里的每个x， 产生出S的所有以x开头的排列， 将对所有x的序列组合起来， 就可以得到S的所有排序。


```
(define (permutations s)
  (if (null? s)
      (list '())
      (flatmap (lambda (x)
                 (map (lambda (p) (cons x p))
                      (permutations (remove x s))
                 )
               )
               s
      )
  )
)
```

```
;定义剔除函数

(define (remove item sequence)
  (filter (lambda (x) (not (= x item)))
           sequence
  )
)
```

验证

```
(permutations (list 1 2 3))
```

`练习题2.40`
请定义过程unique-pairs, 给它整数n， 它产生出序对(i, j) 其中1 <= j < i <= n 请用unique-pairs 去简化上面prime-sum-pairs的定义

```
;前面已经有定义， 抽取出来即可

(define (unique-pairs n)
    (flatmap (lambda (i)
                 (map (lambda (j) (list i j))
                      (enumerate-interval 1 (- i 1))))
             (enumerate-interval 1 n)))

;重新定义
(define (prime-sum-pairs n)
    (map make-pair-sum
         (filter prime-sum? (unique-pairs n))))
         

```

`练习题2.41`

请写出一个过程， 他能产生出所有小于给定整数n的正的相异整数i, j 和k的有序三元组， 使得每个三元组的三个元之和等于给定的整数s

todo 参考其他实现方式： https://sicp.readthedocs.io/en/latest/chp2/41.html



`练习题2.42`

八皇后问题

```
(define (queens board-size)
  (define (queen-cols k)
     (if (= k 0)
         (list empty-board)
         (filter
           (lambda (positions) (safe? k positions))
           (flatmap
              (lambda (rest-of-queens)
                 (map (lambda (new-row)
                        (adjoin-position new-row k rest-of-queens)
                      )
                      (enumerate-interval 1 board-size)
                 
                 )
              )
              (queen-cols (- k 1))
           )
         )
     )
  )
)
```

```
;定义列 加入格局集合中

(define (adjoin-position new-row k rest-of-queens)
    (cons new-row rest-of-queens)
)

;定义空格局

(define (empty-board)
  (list '())
)

;定义k列皇后，相对其他的列皇后是否是安全的， 我们只需要检查新皇后是否安全， 假设其他皇后已经安全
;核心问题： 斜对脚线， 直上 直下问题
(define (safe? k position)
    (iter-check (car position) 
                (cdr position)
                 1))

(define (iter-check row-of-new-queen rest-of-queens i)
    (if (null? rest-of-queens)  ; 下方所有皇后检查完毕，新皇后安全
        #t
        (let ((row-of-current-queen (car rest-of-queens)))
            (if (or (= row-of-new-queen row-of-current-queen)           ; 行碰撞
                    (= row-of-new-queen (+ i row-of-current-queen))     ; 右下方碰撞
                    (= row-of-new-queen (- row-of-current-queen i)))    ; 左下方碰撞
                #f
                (iter-check row-of-new-queen 
                            (cdr rest-of-queens)    ; 继续检查剩余的皇后
                            (+ i 1))))))            ; 更新步进值
```

`2.2.4 实例： 一个图形语言`

本章节将介绍一种用于画图形的简单语言， 以展示数据抽象和闭包的威力， 其中也以一种非常本质的方式使用了高阶过程。 这一语言的设计就是为了很容易的作出一些模式， 例如2-9中所示的那类图形， 它们是由某些元素的重复出现而构成， 这些元素可以变形或者改变大小，

```
图形语言

在描述一种语言时， 应该将注意力集中到语言的基本源语， 它的组合手段以及它的抽象手段。 这是最重要的。 这里的工作也将按照同样的框架进行。

这一图形语言的优美之处， 部分就在于语言中只有一种元素， 称为画家(painter), 一个画家将画出一个图像， 这种图像可以变形或者改变大小， 以便能正好放到某个指定的平行四边形框架里， 举例来说， 这里由一个称为wave的基本画家， 它能作出如图 2-10所示的折线图， 而所作出的图画实际形状依赖于具体的框架----图2-10里的四个图像都是由同一个画家wave产生， 却是相对于四个不同的框架。 有些画家比它更精妙。 

为了组合起来有关画像， 我们要用一些可以从给定画家构造出新画家的操作。 例如， 操作beside从两个画家触发， 产生出一个复合型画家， 它将第一个画家的图像画在框架中左边的一半里， 将第二个画家的画像画在第二个画家的图像之下， 有些操作将一个画家转换为另一个画家， 例如，flip-vert从一个画家触发， 产生一个将该画家所画图像上下颠倒画出的画家， 而flip-horiz产生出的画家将原画家的图像左右翻转后画出。

过程定义：
beside: 组合，左右
below: 将一个画家的图像画在第二个画像的图像之下， 即上下组合
flip-vert: 画像上下调换
flip-horiz 左右翻转

;
(define wav2
  (beside wave (flip-vert wave))
)


在按这种方法构造复杂的图像时， 我们利用了一个事实： 画家在有关语言的组合方式下是封闭的； 两个画家的beside或者below还是画家， 因此还可以用它们作为元素去构造更复杂的画家。 就像用 cons 构造起各种表结构一样， 我们所用的数据在组合方式下的闭包性质非常重要， 因为这使得我们能用不多几个操作构造出各种复杂的结构。

一旦能做画家的组合之后， 我们就希望能抽象出典型的画家组合模式， 以便将这种组合操作实现为一些Scheme过程， 也也一位着我们并不需要这种图形语言包含任何特殊抽象机制， 因为组合的方式就是采用普通的Scheme过程。 这样， 对于画家， 我们就自动有了能够作出原来可以对过程做的所有事情。 例如， 我们可以将wave4中的模式抽象出来：

(define (flip-pairs painter)
  (let ((painter2 (beside painter (flip-vert painter))))
       (below painter2 painter2)
  )
)

并将wave4 重新定义为这种模式的实例：

(define wave4 (flipped-pairs wave))

我们也可以定义递归操作。 下面就是一个这样的操作， 它在图形的右边做分割和分支



```

```
(define (right-split painter n)
  (if (= n 0)
    painter
    (let ((smaller (right-split painter (- n 1))))
       (beside painter (below smaller smaller))
    
    )
  )
)
```

通过同时在图形中向上和向右分支， 可以产生出一种平衡的模式

```
(define (corner-split painter n)
  (if (= n 0)
     painter
     (let ( (up (up-split painter (- n 1)))
            (right (right-split painter (- n 1)))
          )
          (let ((top-left (beside up up))
                (bottom-right (below right right))
                (corner (corner-split painter (-n 1)))
               )
               
               (beside (below painter top-left)
                       (below bottom-right corner)
               )
          )
     )
  )
)
```


将某个corner-split的4个拷贝适当的组合起来， 我们就可以得到一种称为square-limit的模式

```
(define (square-limit painter n)
  (let ((quarter (corner-split painter n)))
       (let ((half (beside (flip-horiz quarter) quarter)))
          (below (flip-vert half) half)
       )
  
  )
)
```

练习题2.44 请定义出corner-split里使用过程up-split, 它与right-split类似， 除在其中交换了below和beside的角色之外。


```
(define (right-split painter n)
  (if (= n 0)
    painter
    (let ((smaller (right-split painter (- n 1))))
       (below  painter (beside smaller smaller))
    
    )
  )
)
```


`高阶操作`

除了可以获取组合画家的抽象模式之外， 我们同样可以在高阶上工作， 抽象出画家的各种组合操作的模式。 也就是说， 可以把画家操作看成是操作和描述这些元素的组合方法的元素---写出一些过程， 它们以画家操作作为参数， 创建出各种新的画家操作。

举例来说， flipped-pairs和square-limit两者都将一个画家的四个拷贝安排在一个正方形的模式中， 它们之间的差异仅仅在这些拷贝的旋转角度。 抽象出这种画家组合模式的一种方式是定义下面的过程， 它基于四个单参数的画家操作， 产生出一个画家操作， 这一操作里将用这四个操作去变换一个给定的画家， 并将得到的结果放入一个正方形里。 tl, tr, bl, 和br 分别是应用于左上角， 右上角， 左下角， 和右下角的四个拷贝的变换

```
;tl 左上角
; tr 右上角
; bl 左下角
; br 右小角
(define (square-of-four tl tr bl br)
  (lambda (painter)
    (let ((top (beside (tl painter) (tr painter)))
           (bottom (beside (bl painter) (br painter)))
         )
         (below bottom top)
    )
  )
)
```

操作flipped-pairs可以基于square-of-four定义如下：

(define (flipped-pairs painter)
  (let ((combine4 (square-of-four identity flip-vert identity flip-vert))
       )
       (combine4 painter)
  )
)

而square-limit可以描述为

```
(define (square-limit painter n)
  (let ( (combine4 (square-of-four flip-horiz identity
                                   rotate180  flip-vert
                   )
       )
       (combine4 (corner-split painter n))
  )
)
```



`练习题2.45`

可以将right-split 和up-split 表述为某种广义划分操作的实例， 请定义一个过程split, 它具有如下性质， 求值

```
(define right-split (split beside below))
(define up-split (split below beside))
;这里参考网上例子
```